# Яндекс Музыка

***Описание проекта***

На основе данных сервиса Яндекс Музыки нужно сравнить поведение пользователей двух столиц Москвы и Санкт-Петербурга.

**Цель исследования**

Проверка трех гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.



## Общая информация о данных




In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('yandex_music_project.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'yandex_music_project.csv'

In [ ]:
df.head(10)

In [ ]:
df.info()
# получение общей информации о данных в таблице df

Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

Согласно документации к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.



## Предобработка данных
Исправить названия столбцов на более удобные названия столбцов, потом исключить пропуски. Затем проверка данных на дубликаты.

### Стиль заголовков


In [ ]:
df.columns
# перечень названий столбцов таблицы df

Переименование названий столбцов:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [ ]:
df = df.rename(columns = {'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})

Проверка результата.

In [ ]:
df.columns

### Пропуски значений

Посчёт, сколько в таблице пропущенных значений.

In [ ]:
df.isna().sum()
# подсчёт пропусков

Не все пропущенные значения влияют на исследование. Так в `track` и `artist` пропуски достаточно заменить явными обозначениями.

Но пропуски в `genre` могут помешать сравнению музыкальных вкусов в Москве и Санкт-Петербурге. Поэтому нужно:
* заполнить и эти пропуски явными обозначениями;
* оценить, насколько они повредят расчётам.

Замена пропущенных значений в столбцах `track`, `artist` и `genre` на строку `'unknown'`.

In [ ]:
columns_to_replace = ['track', 'artist', 'genre']
for i in columns_to_replace:
    df[i] = df[i].fillna('unknown')
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

In [ ]:
df.isna().sum()
# подсчёт пропусков

### Дубликаты

Подсчёт явных дубликатов в таблице:

In [ ]:
df.duplicated().sum() #подсчет явных дубликатов с помощью двух методов

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
# удаление явных дубликатов

In [ ]:
df.duplicated().sum() # проверка на отсутствие дубликатов

Избавление от неявных дубликатов в колонке `genre`.

In [ ]:
genre = df['genre']
genre = genre.sort_values()
genre.unique()
# Просмотр уникальных названий жанров

В данных есть следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.


In [ ]:
dublications = ['hip', 'hop', 'hip-hop']
name = 'hiphop'
df['genre'] = df['genre'].replace(dublications, name)
# Устранение неявных дубликатов

In [ ]:
genre = df['genre']
genre = genre.sort_values()
genre.unique()
# Проверка на неявные дубликаты

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Были исправлены заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения заменили на `'unknown'`.


## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что

**пользователи по-разному слушают музыку в Москве и Санкт-Петербурге**.

Проверка этого предположения по данным о трёх днях недели — понедельнике, среде и пятнице.


In [ ]:
df.groupby('city')['city'].count()
# Подсчёт прослушиваний в каждом городе

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.


In [ ]:
df.groupby('day')['day'].count()
# Подсчёт прослушиваний в каждый из трёх дней

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Создание функции `number_tracks()`, которая посчитает прослушивания для заданного дня и города.

In [ ]:
# создание функции
def number_tracks(day, city):
    data = df[df['day'] == day]
    track_list = data[data['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [ ]:
number_tracks('Monday', 'Moscow')
# количество прослушиваний в Москве по понедельникам

In [ ]:
number_tracks('Monday', 'Saint-Petersburg')
# количество прослушиваний в Санкт-Петербурге по понедельникам

In [ ]:
number_tracks('Wednesday', 'Moscow')
# количество прослушиваний в Москве по средам

In [ ]:
number_tracks('Wednesday', 'Saint-Petersburg')
# количество прослушиваний в Санкт-Петербурге по средам

In [ ]:
number_tracks('Friday', 'Moscow')
#количество прослушиваний в Москве по пятницам

In [ ]:
number_tracks('Friday', 'Saint-Petersburg')
# количество прослушиваний в Санкт-Петербурге по пятницам

Создание таблицы с результатами данных

In [ ]:
info = pd.DataFrame(data = [['Москва', 15740, 11056, 15945], ['Санкт-Петербург', 5614, 7003, 5895]],
                    columns = ['city', 'monday', 'wednesday', 'friday'])
info
# Таблица с результатами

**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Проверка второй гипотезе,
**утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие**.

Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [ ]:
moscow_general = df[df['city'] == 'Moscow']



In [ ]:
spb_general = df[df['city'] == 'Saint-Petersburg']



Создание функции `genre_weekday()`, которая возвращает информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени.

In [ ]:
def genre_weekday(df, day, time1, time2):
    genre_df = df[df['day'] == day]
    genre_df = genre_df[genre_df['time'] < time2]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending = False)
    return genre_df_sorted[:10]

Сравнение результатов функции `genre_weekday()` для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [ ]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')


In [ ]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

In [ ]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

In [ ]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: **Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.**  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [ ]:
moscow_genre = moscow_general.groupby('genre')['genre'].count()
moscow_genres = moscow_genre.sort_values(ascending = False)

In [ ]:
moscow_genres.head(10)

In [ ]:
spb_genre = spb_general.groupby('genre')['genre'].count()
spb_genres = spb_genre.sort_values(ascending = False)

In [ ]:
spb_genres.head(10)

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.


## Итоги исследования

Были проверены три гипотезы и установлено:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге.

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

